In [1]:
import nltk
import pandas as pd
from nrclex import NRCLex
from collections import Counter, defaultdict
from textblob import TextBlob
from collections import Counter
from json import load

In [ ]:
nltk.download('punkt')

In [ ]:
!python -m textblob.download_corpora

In [6]:
# Analyse for each LLM
model = 'gemini'
path = '/projects/humansVsLLMs/results/3-shot-generated-responses'
df = pd.read_csv(f"{path}/{model}_generated_responses.csv")  # Replace with your file
response_texts = df['Response'].dropna().astype(str)

In [ ]:
# Analyze goals of real-life leaders
path = '/projects/humansVsLLMs/data'
df = pd.read_csv(f"{path}/goals_leader_with_demographics.csv")  # Replace with your file
response_texts = df['Leader_Action_Plans'].dropna().astype(str)

In [ ]:
# For gender wise comparison
path = '/projects/humansVsLLMs/data'
df = pd.read_csv(f"{path}/data_leaders_with_demographics_semantics.csv")  # Replace with your file
df = df[~df['GenderIdentity'].isin(['Male', 'Female'])]
response_texts_uniqueness = df['firstTaskGoal'].dropna().astype(str).to_list()
print(len(response_texts_uniqueness))
response_texts_belongingness = df['addFirstRelGoal'].dropna().astype(str).to_list()
print(len(response_texts_belongingness))
response_texts = response_texts_belongingness + response_texts_uniqueness
print(len(response_texts))

In [ ]:
# Aggregated containers
all_affect_list = []
all_affect_dict = defaultdict(int)
all_raw_emotions = Counter()
top_emotions_count = Counter()
affect_freqs = defaultdict(float)

# Process each response
for response in response_texts:
    text_object = NRCLex(response)
   
    # Affect list (adds all labels)
    all_affect_list.extend(text_object.affect_list)
    # print(text_object.affect_dict)
    # print(text_object.raw_emotion_scores)
    # print(text_object.top_emotions)

    # Affect dictionary (sum over all keys)
    # for k, v in text_object.affect_dict.items():
        # all_affect_dict[k] += sum(v)

    # Raw emotion scores
    for k, v in text_object.raw_emotion_scores.items():
        all_raw_emotions[k] += v

    # Top emotions (count the most prominent labels)
    for emotion in text_object.top_emotions:
        top_emotions_count[emotion[0]] += 1

    # Affect frequencies (normalized, so we'll average them later)
    for k, v in text_object.affect_frequencies.items():
        affect_freqs[k] += v

# Normalize affect frequencies
num_responses = len(response_texts)
affect_freqs = {k: v / num_responses for k, v in affect_freqs.items()}

# Display results
print("\n Aggregated Affect List:")
print(all_affect_list[:10], "...")  # show sample

print("\n Aggregated Raw Emotion Scores:")
print(dict(all_raw_emotions))

print("\n Aggregated Top Emotions:")
print(dict(top_emotions_count))

print("\n Averaged Affect Frequencies:")
print(affect_freqs)
